In [3]:
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D
from keras.models import load_model
from keras import optimizers
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D
from keras import backend as K

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

In [9]:
import os, re, glob
import cv2
import numpy as np
#from keras.preprocessing import image
from PIL import Image

In [21]:
import joblib
import pickle

In [17]:
def normalization_tool(data):
    data_nor = data.astype('float32') / 255
    
    return data_nor

In [13]:
def split_data(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
    xy = (X_train, X_test, Y_train, Y_test)
    np.save("./img_data.npy", xy)
    X_train, X_test, Y_train, Y_test = np.load('./img_data.npy', allow_pickle = True)
    
    return X_train, X_test, Y_train, Y_test

In [15]:
batch_size = 32
learning_rate = 0.01
epochs = 500

optimizer='Adam'
loss='binary_crossentropy'

In [16]:
def optimizer_set(optimizer, learning_rate):
    if optimizer == 'SGD':
        optimizer = optimizers.SGD(lr=learning_rate, clipnorm=1.)
        
    if optimizer == 'RMSprop':
        optimizer = optimizers.RMSprop(lr=learning_rate, rho=0.9, epsilon=None, decay=0.0)
        
    if optimizer == 'Adam':
        optimizer = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        
    if optimizer == 'Nadam':
        optimizer = optimizers.Nadam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
        
    if optimizer == 'Adagrad':
        optimizer = optimizers.Adagrad(lr=learning_rate, epsilon=None, decay=0.0)
        
    if optimizer == 'Adadelta':
        optimizer = optimizers.Adadelta(lr=learning_rate, rho=0.95, epsilon=None, decay=0.0)
        
    return optimizer

In [26]:
def twentyeightCNNAutoEncoder():
    model = Sequential()
    
    # 1st convolution layer
    model.add(Conv2D(16, (3, 3)  # 16 is number of filters and (3, 3) is the size of the filter.
                 , padding='same', input_shape=(28, 28, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    # 2nd convolution layer
    model.add(Conv2D(2, (3, 3), padding='same'))  # apply 2 filters sized of (3x3)
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    # here compressed version

    # 3rd convolution layer
    model.add(Conv2D(2, (3, 3), padding='same'))  # apply 2 filters sized of (3x3)
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))

    # 4rd convolution layer
    model.add(Conv2D(16, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))

    model.add(Conv2D(1, (3, 3), padding='same'))
    model.add(Activation('sigmoid'))
    model.summary()
    model.save('./model/model.h5')
    
    return model

In [24]:
def ninetysixCNNAutoEncoder():
    model = Sequential()
    
    # 1st convolution layer
    model.add(Conv2D(16, (3, 3)  # 16 is number of filters and (3, 3) is the size of the filter.
                 , padding='same', input_shape=(96, 96, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    # 2nd convolution layer
    model.add(Conv2D(2, (3, 3), padding='same'))  # apply 2 filters sized of (3x3)
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    # here compressed version

    # 3rd convolution layer
    model.add(Conv2D(2, (3, 3), padding='same'))  # apply 2 filters sized of (3x3)
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2), padding='same'))

    # 4rd convolution layer
    model.add(Conv2D(16, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
    
    #5th convolution layer
    model.add(Conv2D(2,(3, 3), padding='same')) # apply 2 filters sized of (3x3)
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
 
    #6th convolution layer
    model.add(Conv2D(16,(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))

    model.add(Conv2D(1, (3, 3), padding='same'))
    model.add(Activation('sigmoid'))
    model.summary()
    model.save('./model/model.h5')
    
    return model

In [25]:
def train_tool(X, Y, batch_size, learning_rate, epochs, optimizer, loss):
    X_train, X_test, Y_train, Y_test = split_data(X, Y) #데이터 분리
    X_train = normalization_tool(X_train) #데이터 정규화
    X_test = normalization_tool(X_test) #데이터 정규화
    
    optimizer = optimizer_set(optimizer, learning_rate) #optimizer 설정
    
    model = model.load_model('./model/model.h5')
    
    #학습(약 7000번 정도 진행)
    model.compile(optimizer=optimizer, loss=loss) #사용자 지정 파라미터(optimizer, loss)
    resulit = model.fit(X_train, X_train, batch_size = batch_size, epochs=epochs, validation_data=(X_test, X_test)) #사용자 지정 파라미터(epochs)
    
    return result

In [22]:
def make_feature_extraction_twentyeight(X):
    X = normalization_tool(X) #데이터 정규화
    
    compressed_layer = 5
    get_3rd_layer_output = K.function([model.layers[0].input],[model.layers[compressed_layer].output])
    compressed = get_3rd_layer_output([X])[0]
    
    #일렬로 늘리기
    compressed = compressed.reshape(compressed.shape[0], compressed.shape[1]*compressed.shape[2]*compressed.shape[3])
    
    return compressed

In [23]:
def make_feature_extraction_ninetysix(X):
    X = normalization_tool(X) #데이터 정규화
    
    compressed_layer = 8
    get_3rd_layer_output = K.function([model.layers[0].input],[model.layers[compressed_layer].output])
    compressed = get_3rd_layer_output([X])[0]
    
    #일렬로 늘리기
    compressed = compressed.reshape(compressed.shape[0], compressed.shape[1]*compressed.shape[2]*compressed.shape[3])
    
    return compressed